# Installs

In [1]:
!python -m venv myenv

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/myenv/bin/python3



In [2]:
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-vhltp

In [3]:
!pip install accelerate peft bitsandbytes transformers trl
!pip install "transformers>=4.45.1"
!pip install tiktoken
!pip install blobfile
!pip install 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_multi-backend-refactor/bitsandbytes-0.44.1.dev0-py3-none-manylinux_2_24_x86_64.whl'
!pip install -U bitsandbytes
!pip install numpy==1.23.5
# !pip install --force-reinstall 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_multi-backend-refactor/bitsandbytes-0.44.1.dev0-py3-none-manylinux_2_24_x86_64.whl'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.45.0
    Uninstalling bitsandbytes-0.45.0:
      Successfully uninstalled bitsandbytes-0.45.0
  Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl.metadata (2.9 kB)
Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl (69.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.44.1.dev0+9315692
    Uninstalling bitsandbytes-0.44.1.dev0+9315692:
      Successfully uninstalled

# Login and Imports

In [1]:
from huggingface_hub import notebook_login
notebook_login()


In [2]:
# Installing More Dependencies
import torch
import numpy
import pandas as pd
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [3]:
def get_model_and_tokenizer(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
  )
  model = AutoModelForCausalLM.from_pretrained(
      model_id, quantization_config=bnb_config, device_map="auto"
  )
  model.config.use_cache=False
  model.config.pretraining_tp=1
  return model, tokenizer

# Prep Base Llama Model

In [7]:
model_id="meta-llama/Llama-3.1-8B"

In [8]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [9]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=60,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

# Fine Tune Llama

In [10]:
output_model="llama3.1-8B-FineTuneTranslationRefinement-WithSource"

## Format Fine Tune Data

In [16]:
import pandas as pd
def prepare_train_datav2(data):
    # Convert the data to a Pandas DataFrame
    # Create a new column called "text"
    data["text"] = data[["translation_text", "german_text", "actual_translation"]].apply(lambda x: "<|im_start|>user\nFix the following German to English translation:\nGerman: " + x["german_text"] + "\nEnglish Translation: " + x["translation_text"] + "\n\nHere is the fixed translation:<|im_end|>\n<|im_start|>assistant\n" + x["actual_translation"]+ "<|im_end|>", axis=1)
    # Create a new Dataset from the DataFrame
    data = Dataset.from_pandas(data)
    return data

In [17]:
data_df = pd.read_csv('100000_translation_sample.csv')
data = prepare_train_datav2(data_df)

In [18]:
print(data[4]['text'])

<|im_start|>user
Fix the following German to English translation:
German: Die vorläufige, schreckliche Bilanz in El Salvador lautet zurzeit bereits: 350 Tote, 1 200 Vermisste, eine vollständig verwüstete Region und Tausende zerstörter Häuser im gesamten Land.
English Translation: The preliminary, terrible balance in El Salvador is currently: 350 dead, 1,200 missing, a completely destroyed region and thousands of destroyed houses throughout the country.

Here is the fixed translation:<|im_end|>
<|im_start|>assistant
The latest, provisional, figures for victims in El Salvador are already very high. There are 350 people dead, 1 200 people missing, the area is completely devastated and thousands of homes have been destroyed throughout the country.<|im_end|>


## Preparing LoRa

In [19]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM"
    )

In [20]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="steps",
        save_steps = 100,
        logging_steps=10,
        num_train_epochs=.5,
        fp16=True,
        push_to_hub=True,
        resume_from_checkpoint="path/to/checkpoint"
    )


In [21]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=2048
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

## Fine Tune!

In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.973700
20,1.244900
30,1.086700
40,1.054600
50,1.044500
60,1.049400
70,1.025000
80,1.038100
90,0.999500
100,0.967700


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.50 GiB. GPU 0 has a total capacity of 39.56 GiB of which 918.81 MiB is free. Process 10367 has 38.66 GiB memory in use. Of the allocated memory 36.79 GiB is allocated by PyTorch, and 1.36 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Model Evaluation

In [4]:
import pandas as pd

In [5]:
finetune_model_name = 'palverde1/llama3.1-8B-FineTuneTranslationRefinement-WithSource'
model, tokenizer = get_model_and_tokenizer(finetune_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [6]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=10,temperature=.5,repetition_penalty=1.2,
      max_new_tokens=30,pad_token_id=tokenizer.eos_token_id
  )
  inputs = tokenizer(user_input, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  return theresponse



## Run Validation Translations Through Model

In [7]:
val_df = pd.read_csv('1500_validation_translations.csv')

In [8]:
val_df['input_text'] = val_df[["translation_text", "german_text"]].apply(lambda x: "<|im_start|>user\nFix the following German to English translation:\nGerman: " + x["german_text"] + "\nEnglish Translation: " + x["translation_text"] + "\n\nHere is the fixed translation:<|im_end|>\n", axis=1)


In [11]:
print(val_df['input_text'][4])

<|im_start|>user
Fix the following German to English translation:
German: Die älteste offizielle Karte Münchens fördert spannende Geschichten zu Tage.
English Translation: The oldest official map of Munich promotes exciting stories to days.

Here is the fixed translation:<|im_end|>



In [12]:
val_llama_cleaned_transcripts = []
for i in range(len(val_df['input_text'])): #
    llama_cleaned_transcript = generate_response(user_input=val_df['input_text'][i])
    val_llama_cleaned_transcripts.append(llama_cleaned_transcript)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
import json
file_name_j = f"val_llama_3.1_v2_output.json"
with open(file_name_j, "w") as f:
    json.dump(val_llama_cleaned_transcripts, f)

In [14]:
val_df['llama_3.1_v2_translation'] = val_llama_cleaned_transcripts

In [22]:
val_llama_cleaned_transcripts[:4]

['<|im_start|>user\nFix the following German to English translation:\nGerman: München 1856: Vier Karten, die Ihren Blick auf die Stadt verändern\nEnglish Translation: Munich 1856: Four maps that change your view of the city\n\nHere is the fixed translation:<|im_end|>\n<|im_start|>assistant\nMunich in 1856 - four cards which alter your vision of the town.<|im_end|',
 '<|im_start|>user\nFix the following German to English translation:\nGerman: Eine Irren-Anstalt, wo sich heute Jugendliche begegnen sollen.\nEnglish Translation: This is a place where young people are to meet today.\n\nHere is the fixed translation:<|im_end|>\n<|im_start|>assistant\nA lunatic asylum which should now house youngsters.<|im_end|>',
 '<|im_start|>user\nFix the following German to English translation:\nGerman: Eine Gruftkapelle, wo nun für den S-Bahn-Tunnel gegraben wird.\nEnglish Translation: A mining chapel, where it is now buried for the S-Bahn tunnel.\n\nHere is the fixed translation:<|im_end|>\n<|im_start|>

## Score the Translations

In [20]:
import evaluate
import numpy as np

def bleurt_score_translations(model_output, llama):
  if llama:
    model_output = [translation.split('\n<|im_start|>assistant\n')[1].split('<|im_end|>')[0] for translation in model_output]
  bleurt = evaluate.load("bleurt", module_type="metric")
  results = bleurt.compute(predictions=model_output, references=val_df['actual_translation'])
  return results

In [21]:
# BLEURT FOR FINETUNNED LLAMA_3.1 8B With German Source
llama_3_1_v2_model_output = val_df['llama_3.1_v2_translation']
llama_3_1_v2_bleurt_scores = bleurt_score_translations(llama_3_1_v2_model_output, llama = True)['scores']
print("Mean BLEURT Score for Finetunned Llama3.1 8B V2: " + str(np.round(np.mean(llama_3_1_v2_bleurt_scores), 5)))

Mean BLEURT Score for Finetunned Llama3.1 8B V2: -0.22415
